In [1]:
import pandas as pd
import numpy as np
import os.path
from keras.applications import ResNet50
from keras.optimizers import Adam
import cv2
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [4]:
train["Class"].unique()

array(['Food', 'misc', 'Attire', 'Decorationandsignage'], dtype=object)

In [5]:
train['Image'].values

array(['image7042.jpg', 'image3327.jpg', 'image10335.jpg', ...,
       'image4975.jpg', 'image1222.jpg', 'image9635.jpg'], dtype=object)

In [6]:
IMG_SIZE = 256

def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img

In [7]:
TRAIN_PATH = 'Train Images/'
TEST_PATH = 'Test Images/'

In [8]:
train_img = []
for img_name in tqdm(train['Image'].values):
    train_img.append(read_img(TRAIN_PATH + img_name))

100%|████████████████████████████████████████████████████████████████████████████| 5983/5983 [00:04<00:00, 1332.85it/s]


In [9]:
type(train_img)

list

In [10]:
x_train = np.array(train_img, np.float32) / 255

In [11]:
print(x_train.shape)

(5983, 256, 256, 3)


In [12]:
label_list = train['Class'].tolist()
label_numeric = {k: v+1 for v, k in enumerate(set(label_list))}
y_train = [label_numeric[k] for k in label_list]
y_train = np.array(y_train)

In [13]:
print(y_train.shape)

(5983,)


In [14]:
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [15]:
y_train = to_categorical(y_train)

In [16]:
print(y_train.shape)

(5983, 5)


In [17]:
base_model =   applications.ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

C:\Users\harka\Miniconda3\envs\tensorflow\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [18]:
## set model architechture 
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dropout(0.5))
add_model.add(Dense(512, activation='relu'))
add_model.add(Dropout(0.4))
add_model.add(Dense(y_train.shape[1], activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.8),
              metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [19]:
batch_size = 5            
epochs = 25                   

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_datagen.fit(x_train)

In [20]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs
)


Epoch 1/25
1196/1196 [==============================] - 428s 358ms/step - loss: 1.3863 - accuracy: 0.5258
Epoch 2/25
1196/1196 [==============================] - 411s 344ms/step - loss: 0.9109 - accuracy: 0.6296
Epoch 3/25
1196/1196 [==============================] - 411s 344ms/step - loss: 0.8247 - accuracy: 0.6751
Epoch 4/25
1196/1196 [==============================] - 409s 342ms/step - loss: 0.7585 - accuracy: 0.6997
Epoch 5/25
1196/1196 [==============================] - 410s 342ms/step - loss: 0.6984 - accuracy: 0.7276
Epoch 6/25
1196/1196 [==============================] - 404s 337ms/step - loss: 0.6780 - accuracy: 0.7364
Epoch 7/25
1196/1196 [==============================] - 424s 355ms/step - loss: 0.6452 - accuracy: 0.7524
Epoch 8/25
1196/1196 [==============================] - 405s 339ms/step - loss: 0.6007 - accuracy: 0.7681
Epoch 9/25
1196/1196 [==============================] - 404s 337ms/step - loss: 0.5738 - accuracy: 0.7757
Epoch 10/25
1196/1196 [=======================

In [21]:
model.save_weights("modelgala52.h5")

In [22]:
test_img = []
for img_name in tqdm(test['Image'].values):
    test_img.append(read_img(TEST_PATH + img_name))

100%|█████████████████████████████████████████████████████████████████████████████| 3219/3219 [00:13<00:00, 239.74it/s]


In [23]:
x_test = np.array(test_img, np.float32) / 255 

In [24]:
predictions = model.predict(x_test)

In [25]:
# get labels
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in label_numeric.items()}
pred_labels = [rev_y[k] for k in predictions]

In [28]:
sub = pd.DataFrame({'Image': test.Image, 'Class': pred_labels})
sub = sub[['Image', 'Class']]
filename = 'solution52 .csv'
sub.to_csv(filename, index=False) 
sub.head()

,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,misc
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire
